In [1]:
from IPython.display import display_html
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Toggle Code</button>""", raw=True)

Toggle Code

# OpenStreetMap
***Data Wrangling with mongoDB by NK Zhehua Zou***
  
Map Area: San Jose, CA, United States  
https://mapzen.com/data/metro-extracts/metro/san-jose_california/  
  
***Table of Contents***
1. Data Audit
2. Problems Encountered in the Map  
Abbreviated Street Names  
Postal Codes  
3. Data Overview  
4. Additional Ideas  
Contributor statistics and gamification suggestion  
Additional data exploration using MongoDB  
5. Conclusion

# 1. Data Audit

In [2]:
# Load packages and libraries
import sys
sys.path.append("script/")
import xml.etree.cElementTree as ET
import re

### cleaning ###
from collections import defaultdict
import string

### osm to json ###
from pymongo import MongoClient
import os
import codecs
import json

In [3]:
# Load data
# This data just a sample for code testing
# Please read html file if you want to reviewed entire analysis.
data = 'data/sample.osm'

### Tags
Parse through the San Jose dataset with ElementTree and count the number of unique element types to get an overall understanding of the data by using count_tags function.

In [4]:
# Parse through the data with ElementTree
def count_tags(data):
    tags={}
    for event, elem in ET.iterparse(data):
        if elem.tag in tags:
            tags[elem.tag]+=1
        else:
            tags[elem.tag]=1
    return tags

count_tags(data)

{'bounds': 1,
 'member': 115,
 'nd': 21564,
 'node': 13332,
 'osm': 1,
 'relation': 21,
 'tag': 7128,
 'way': 3721}

### Keys Type
*** For the follinwg function: key_type & process_key. We check the "k" value for each. ***  
"lower", for tags that contain only lowercase letters and are valid.  
"lower_colon", for otherwise valid tags with a colon in their names.  
"problemchars", for tags with problematic characters.

In [5]:
# Count of each of three tag categories in a dictionary with re
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
# This regex represents invalid MongoDB characters for keys.
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == 'tag':
        if re.match(lower,element.get('k'))!=None:
            keys['lower']+=1
        elif re.match(lower_colon,element.get('k'))!=None:
            keys['lower_colon']+=1
        elif re.match(problemchars,element.get('k'))!=None:
            keys['problemchars']+=1
        else:
            keys['other']+=1
    return keys

def process_key(data):
    keys = {'lower': 0, 'lower_colon': 0, 'problemchars': 0, 'other': 0}
    for _, element in ET.iterparse(data):
        keys = key_type(element, keys)
    return keys

process_key(data)

{'lower': 6751, 'lower_colon': 363, 'other': 14, 'problemchars': 0}

### Users

In [6]:
# get users info with ElementTree
def process_people(data):
    users = set()
    for _, element in ET.iterparse(data):
        for e in element:
            if 'uid' in e.attrib:
                users.add(e.attrib['uid'])
    return users

number_contributors = len(process_people(data))

print str(number_contributors) + ' peoples invovlved in the map editing.'

94 peoples invovlved in the map editing.


# 2. Problems Encountered in the Map
After initially downloading a small sample size of the San Jose area and running it, I noticed three main problems with the data, which I will discuss in the following order:  
1) Abbreviated street names ('Branham Ln')  
2) Inconsistent postal codes ('CA950543', '95014-1899')

### Abbreviated Street Names
Once the data was imported to MongoDB, some basic querying revealed street name abbreviations. I updated all substrings in problematic address strings, such that 'Branham Ln' becomes 'Branham Lane'.

1) The main problem we encountered in this dataset come from the street name abbreviation inconsistency. We build the regex matching the last element in the string, where usually the street type is based. Then we come up with a list of mapping that need not to be cleaned.  
2) audit_street_type function search the input string for the regex. If there is a match and it is not within the 'expected' list, add the match as a key and add the string to the set.  
3) is_street_name function looks at the attribute k if k='addre:street'.  
4) audit functio will return the list that match previous two functions.  
5) After that, we would do a pretty print the output of the audit. With the list of all the abbreviated street types we can understand and fill-up our 'mapping' dictionary as a preparatio to convert these street name into proper form.  
6) update_name is the last step of the process, which take the old name and update them with a better name.

### Zip Codes
Postal code strings posed a different sort of problem, forcing a decision to strip all leading and trailing characters before and after the main 5-digit zip code. This effectually dropped all leading state characters (as in 'CA950543') and 4-digit zip code extensions following a hyphen ('95014-1899'). This 5-digit constriction benefits MongoDB aggregation calls on postal codes.  
1) Although most of the zip code is correct, there're still a lot of zip code with incorrect 5 digit formats. We will process it like update street name.  
2 )The output of the clean zip code is summarised below. There are the format of 5 digits or string 'None'.

# 3. Data Overview
This section contains basic statistics about the dataset and the MongoDB queries used to gather them.  
  
### Preparing for MongoDB by converting XML to JSON
In order to transform the data from XML to JSON, we need to follow these rules:  
1) Process only 2 types of top level tags: "node" and "way"  
2) All attributes of "node" and "way" should be turned into regular key/value pairs, except:   attributes in the CREATED array should be added under a key "created", attributes for latitude and longitude should be added to a "pos" array, for use in geospacial indexing. Make sure the values inside "pos" array are floats and not strings.  
3) If second level tag "k" value contains problematic characters, it should be ignored  
4) If second level tag "k" value starts with "addr:", it should be added to a dictionary "address"  
5) If second level tag "k" value does not start with "addr:", but contains ":", you can process it same as any other tag.  
6) If there is a second ":" that separates the type/direction of a street, the tag should be ignored  
After all the cleaning and data transformation are done, we would use last function process_map and convert the file from XML into JSON format

### File sizes

In [7]:
client = MongoClient()
db=client.project

In [8]:
print 'The original OSM file is ' + str(os.path.getsize(data)/1.0e6) + ' MB'

The original OSM file is 3.329866 MB


In [9]:
print 'The JSON file is ' + str(os.path.getsize(data + '.json')/1.0e6) + ' MB'

The JSON file is 3.795992 MB


In [10]:
# Number of documents, we defined it for next section.
number_document = db.doc.find().count()
print 'The number of documents is ' + str(number_document)

The number of documents is 17053


In [11]:
# Number of nodes
print 'The number of node is ' + str(db.doc.find({'type':'node'}).count())

The number of node is 13332


In [12]:
# Number of ways
print 'The number of way is ' + str(db.doc.find({'type':'way'}).count())

The number of way is 3721


In [13]:
# Number of unique users, we defined it for next section.
number_unique_users = len(db.doc.distinct('created.user'))
print 'The number of unique users is ' + str(number_unique_users)

The number of unique users is 91


In [14]:
# Top 1 contributing user
cursor = db.doc.aggregate([{'$group':{'_id':'$created.user', 'count':{'$sum':1}}}, {'$sort':{'count':-1}}, {'$limit':1}])
for res in cursor:
    user1=res['_id']
    user1_count=res['count']
print 'The first contributor is ' + user1 + ' with '+ str(user1_count) + ' contributions.'

The first contributor is TELCOM IP with 4921 contributions.


In [15]:
# Number of users appearing only once (having 1 post), we defined it for next section.
user_once=db.doc.aggregate([{'$group':{'_id':'$created.user', 'count':{'$sum':1}}}, 
                       {'$sort':{'count':1}},
                       {'$match':{'count':1}},
                       {'$group':{'_id':'null','total':{'$sum':'$count'}}}
                        ])
for res in user_once:
    number_user_once=res['total']

print 'There is ' + str(number_user_once) + ' users appearing only once.'

There is 22 users appearing only once.


# 4. Additional Ideas
### Contributor statistics and suggestion
According to these results below, we found unbelievable truth.  
1) Best contributor gave 28% documents, greater than 1/4 of total contributions.  
2) Three contributors also over half of total contributions, it means top 2 and top 3 contributors are far behind top 1 contributors.  
3) Just 20 contributors already gave 96% of total documents, it means rest of people almost have not any contributors in here even if still have 24% contributors gave one post.  
4) Eeery contributor shall gave 187 documents by average contribution, but most of people can't close to this number.  
5) What incentives should we increase? Perhaps we can refer to the experience of waze, which is a great application for navigation app. We can be divided different levels according to contribution, each level users will enjoy different privileges, badges and rewards.

In [16]:
def topn_contrib(n, user=False):
    if user==True:
        topuser=db.doc.aggregate([{'$group':{'_id':'$created.user', 'count':{'$sum':1}}}, 
                                 {'$sort':{'count':-1}}, {'$limit':n}
                                 ])
        top_n_users=[]
        for res in topuser:
            top_n_users.append(res['_id'])

    top_n_contrib=db.doc.aggregate([{'$group':{'_id':'$created.user', 'count':{'$sum':1}}}, 
                         {'$sort':{'count':-1}}, {'$limit':n},
                         {'$group':{'_id':'$created.user','total':{'$sum':'$count'}}}
                        ])

    for res in top_n_contrib:
        top_n_contrib_count=res['total']

    percent_contrib_topn=(top_n_contrib_count*100)/number_document
    
    if user==True:
        return top_n_users,percent_contrib_topn
    else:
        return percent_contrib_topn

In [17]:
top1,top1_percent_contrib=topn_contrib(1,user=True)
print 'Top1 Contributor is ' + str(top1) + ', contribution percentage is ' + str(top1_percent_contrib) + '%.'

Top1 Contributor is [u'TELCOM IP'], contribution percentage is 28%.


In [18]:
top3,top3_percent_contrib=topn_contrib(3,user=True)
print 'These contributors: ' + str(top3) + ' have ' + str(top3_percent_contrib) + '% contribution rate in this area.'

These contributors: [u'TELCOM IP', u'negro', u'Diego Sanguinetti'] have 55% contribution rate in this area.


In [19]:
top20,top20_percent_contrib=topn_contrib(20, user=True)
print 'Contribution percentage from top 20 users is ' + str(top20_percent_contrib) + '%.'

Contribution percentage from top 20 users is 96%.


In [20]:
percent_user_1post=(number_user_once*100)/number_unique_users
print str(percent_user_1post) + '% of users contribute with one post.'

24% of users contribute with one post.


In [21]:
average = number_document/number_unique_users
print 'Average number of documents per contributor is ' + str(average)

Average number of documents per contributor is 187


### Additional data exploration using MongoDB queries
1) 17053 people living in this area.  
2) We found most amenities are bus stations, gas stations (fuel) and schools in this area. Let's explorating them.  
3) I am a bit suprised for this result, there has many long distance bus station in this small area. I though there has many caltrain or city bus stations.  
4) It looks like Grifo has most gas stations in this area, no much suprised for this result, but I can't belive only 1 Vista and have not some popular gas station like Shell in here.  
5) Pescador is the most popular restaurant in this area, they have 2 restaurant in here. After google it, I found this is a fish restaurant and keep a lower rate 3 in 5 stars. Interesting, they don't have delicious food for residents?

In [22]:
population = db.doc.aggregate([{'$group':{'_id':'population', 'count':{'$sum':1}}},
                    {'$sort':{'count':-1}}, {'$limit':10}])

print list(population)

[{u'count': 17053, u'_id': u'population'}]


In [23]:
# Let's check the number of amenity first
amenity = db.doc.aggregate([{'$match':{'amenity':{'$exists':1}}},
                               {'$group':{'_id':'$amenity', 'count':{'$sum':1}}},
                               {'$sort':{'count':-1}}])
for doc in amenity:
    print doc

{u'count': 22, u'_id': u'bus_station'}
{u'count': 12, u'_id': u'fuel'}
{u'count': 12, u'_id': u'school'}
{u'count': 7, u'_id': u'restaurant'}
{u'count': 5, u'_id': u'hospital'}
{u'count': 5, u'_id': u'place_of_worship'}
{u'count': 4, u'_id': u'townhall'}
{u'count': 4, u'_id': u'marketplace'}
{u'count': 4, u'_id': u'university'}
{u'count': 4, u'_id': u'veterinary'}
{u'count': 3, u'_id': u'fire_station'}
{u'count': 2, u'_id': u'bank'}
{u'count': 2, u'_id': u'bar'}
{u'count': 2, u'_id': u'cafe'}
{u'count': 1, u'_id': u'doctors'}
{u'count': 1, u'_id': u'police'}
{u'count': 1, u'_id': u'community_centre'}
{u'count': 1, u'_id': u'post_office'}
{u'count': 1, u'_id': u'clinic'}
{u'count': 1, u'_id': u'public_building'}
{u'count': 1, u'_id': u'parking'}


In [24]:
bus_station = db.doc.aggregate([{'$match':{'amenity':{'$exists':1}, 'amenity':'bus_station'}},
                               {'$group':{'_id':'$name', 'count':{'$sum':1}}},
                               {'$sort':{'count':-1}}])
for doc in bus_station:
    print doc

{u'count': 2, u'_id': None}
{u'count': 2, u'_id': u'Emtrafesac'}
{u'count': 1, u'_id': u'Ku\xe9lap'}
{u'count': 1, u'_id': u'exclusiva'}
{u'count': 1, u'_id': u'Emtrafesa'}
{u'count': 1, u'_id': u'cruz del sur'}
{u'count': 1, u'_id': u'Transportes Chiclayo'}
{u'count': 1, u'_id': u'IttsaBus'}
{u'count': 1, u'_id': u'Tepsa'}
{u'count': 1, u'_id': u'americas express'}
{u'count': 1, u'_id': u'Terminal Flores Hermanos'}
{u'count': 1, u'_id': u'Transportes Linea'}
{u'count': 1, u'_id': u'Oltursa'}
{u'count': 1, u'_id': u'Collectivos'}
{u'count': 1, u'_id': u'Transportes Acunta'}
{u'count': 1, u'_id': u'El Cumbe'}
{u'count': 1, u'_id': u'Burga Express'}
{u'count': 1, u'_id': u'Terminal Nort - Terminal North'}
{u'count': 1, u'_id': u'Transportes JEANBUCA'}
{u'count': 1, u'_id': u'Movil Tours'}


In [25]:
gas_station = db.doc.aggregate([{'$match':{'amenity':{'$exists':1}, 'amenity':'fuel'}},
                    {'$group':{'_id':'$name', 'count':{'$sum':1}}},
                    {'$sort':{'count':-1}}])

for doc in gas_station:
    print doc

{u'count': 2, u'_id': None}
{u'count': 2, u'_id': u'Primax'}
{u'count': 1, u'_id': u'Vista Alegre'}
{u'count': 1, u'_id': u'Pecsa'}
{u'count': 1, u'_id': u'Pesca'}
{u'count': 1, u'_id': u'Grifo San Antonio'}
{u'count': 1, u'_id': u'Grifos Gran Prix'}
{u'count': 1, u'_id': u'Grifo Le\xf3n de Oro'}
{u'count': 1, u'_id': u'Repsol'}
{u'count': 1, u'_id': u'Terminal EPSEL'}


In [26]:
restaurant = db.doc.aggregate([{'$match':{'amenity':{'$exists':1}, 'amenity':'restaurant'}}, 
                    {'$group':{'_id':'$name', 'count':{'$sum':1}}},
                    {'$sort':{'count':-1}}, {'$limit':10}])

for doc in restaurant:
    print doc

{u'count': 2, u'_id': u'El Paladar'}
{u'count': 1, u'_id': u'Mi Tia'}
{u'count': 1, u'_id': u'D Chota'}
{u'count': 1, u'_id': u'GIZA'}
{u'count': 1, u'_id': u'Hebron'}
{u'count': 1, u'_id': u'El Pescador'}


# 5. Conclusion
1) The map about the city of San Jose is relatively clean so I could retrieve some interesting content. But still the data is not entirely clean.  
2) The data contains some mistakes or different references for the same feature. So I had to clean the data programmatically for the street and the postal codes.  
3) When we audit the data, it was very clear that although there are minor error caused by human input, the dataset is fairly well-cleaned. Considering there're hundreds of contributors for this map, there is a great numbers of human errors in this project. I'd recommend a srtuctured input form so everyone can input the same data format to reduce this error.  
4) We can incentivize users by gamify the contribution process, then we can create a recommendation engine to leverage these data (eg. restaurant recommendation, building, etc).  
5) OpenStreetMaps is an open source project, there're still a lot of areas left unexplored as people tend to focus on a certain key areas and left other part outdated. Since each node has a coordinate (lattitude & longtitude), we can resolve this issue by cross-referencing/cross-validating missing data from other database like Google API.